In [ ]:

import boto3

# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')

s3 = boto3.client('s3')
glue = boto3.client('glue')
athena = boto3.client('athena')
sts = boto3.client('sts')
iam_client = boto3.client('iam')

session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id


In [100]:
agent_name = "aws-operations-agent"
agent_alias_name = "workshop-alias"
schema_name = 'text_to_sql_openapi_schema.json'

suffix = f"{region}-{account_id}"
idleSessionTTLInSeconds=3600

bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'

In [ ]:
import json
# Reading the JSON file back (to verify)
with open('agent_info.json', 'r') as json_file:
    loaded_data = json.load(json_file)

print("Data read from file:", loaded_data)

agent_id=loaded_data.get('agentId')
action_group_id=loaded_data.get('actionGroupId')
agentAliasId=loaded_data.get('agentAliasId')
action_group_name=loaded_data.get('actionGroupName')

In [102]:
bucket_name="<your bucket name>"
bucket_key="bedrock/text_to_sql_openapi_schema.json"

In [ ]:
try:
    resp=lambda_client.get_function(FunctionName=lambda_name)
    print(resp['Configuration']['FunctionArn'])
    FunctionArn=resp['Configuration']['FunctionArn']

    response = bedrock_agent_client.update_agent_action_group(
       agentId=agent_id,
       agentVersion='DRAFT',
       actionGroupId= action_group_id,
       actionGroupName=action_group_name,
       actionGroupExecutor={
           'lambda': FunctionArn
       },
       apiSchema={
           's3': {
               's3BucketName': bucket_name,
               's3ObjectKey': bucket_key
           }
       },
       actionGroupState='DISABLED',
    )
except Exception as e:
    print('can not delete',str(e) )


In [ ]:
try: 
    action_group_deletion = bedrock_agent_client.delete_agent_action_group(
       agentId=agent_id,
       agentVersion='DRAFT',
       actionGroupId= action_group_id
    )
except Exception as e:
    print('can not delete',str(e) )

try:
    agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agentAliasId
    )
except Exception as e:
    print('can not delete',str(e) )

try:
    agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
    )
except Exception as e:
    print('can not delete',str(e))

In [ ]:

try:
    # Delete Lambda function
    lambda_client.delete_function(
        FunctionName=lambda_name
    )
except Exception as e:
    print('can not delete',str(e))



In [95]:
import boto3
from botocore.exceptions import ClientError

def delete_iam_role(role_name):
    """
    Deletes an IAM role.

    :param role_name: The name of the role to delete.
    """
    iam = boto3.client('iam')

    try:
        # First, detach all policies from the role
        attached_policies = iam.list_attached_role_policies(RoleName=role_name)
        for policy in attached_policies['AttachedPolicies']:
            iam.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
            print(f"Detached policy {policy['PolicyArn']} from role {role_name}")

        # Delete inline policies
        inline_policies = iam.list_role_policies(RoleName=role_name)
        for policy_name in inline_policies['PolicyNames']:
            iam.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            print(f"Deleted inline policy {policy_name} from role {role_name}")
        # Finally, delete the role
        iam.delete_role(RoleName=role_name)
        print(f"Successfully deleted role {role_name}")

    except ClientError as e:
        print(f"Error deleting role {role_name}: {e}")
        raise




In [ ]:

# Usage
try:
    delete_iam_role(lambda_role_name)
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
# Usage
try:
    delete_iam_role(agent_role_name)
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [109]:


def delete_policy_by_name(policy_name):
    # List all policies
    paginator = iam_client.get_paginator('list_policies')
    for response in paginator.paginate(Scope='Local'):
        for policy in response['Policies']:
            if policy['PolicyName'] == policy_name:
                policy_arn = policy['Arn']
                # Delete the policy by ARN
                try:
                    iam_client.delete_policy(PolicyArn=policy_arn)
                    print(f"Policy '{policy_name}' deleted successfully.")
                    return
                except Exception as e:
                    print(f"Error deleting policy '{policy_name}':", e)
                    return
    print(f"Policy '{policy_name}' not found.")


In [ ]:
try:
    # Example usage
    delete_policy_by_name(bedrock_agent_bedrock_allow_policy_name)
    delete_policy_by_name(bedrock_agent_s3_allow_policy_name)
except Exception as e:
    print(f"An error occurred: {str(e)}")